<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/ask_6_mixed_pubmed_cadec_ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [ ]:
# training data set

train_csv = path/'AskAPatient.fold-6.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-6.validation.csv",header=None)
valid.head()

,0,1
0,2,heel pain
1,2,heels hurt
2,4,black and blue marks
3,5,fast and slow heartbeat
4,5,increased heart rate


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-6.test.csv",header=None)
test.head()

,0,1
0,2,left heel pain
1,5,tachycardia
2,8,ARTHRITIS
3,8,arthrites
4,8,arthritis


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'pubmed_cadec_askpatient_mixed.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [ ]:
data_lm.train_ds.vocab.itos

In [22]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [23]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [24]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.806585,3.466722,0.436493,30:37


In [25]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.382773,3.235633,0.457803,30:35


In [26]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.213276,3.205427,0.461234,30:42


In [27]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.167798,3.184791,0.463425,30:35


In [28]:
learn.save_encoder('pubmed_cadec_askpatient_mixed_fold6_first')

In [29]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_askpatient_mixed_fold6_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.755969,3.271973,0.427807,01:02


In [30]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.778125,2.698616,0.525401,01:17


In [31]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.217008,2.333318,0.593583,03:07


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.799508,1.953059,0.645722,03:06
1,1.621215,1.837850,0.673797,03:08


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.447675,1.657302,0.700535,03:01
1,1.257671,1.549605,0.717914,03:11


In [34]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.064949,1.486600,0.728610,03:11
1,0.969959,1.388304,0.747326,03:09


In [35]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.900236,1.366145,0.748663,03:12
1,0.800560,1.259783,0.770053,03:13


In [36]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.741935,1.271902,0.767380,02:56
1,0.664700,1.172468,0.772727,03:12


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.632378,1.188473,0.775401,03:01
1,0.613061,1.138912,0.783422,02:54
2,0.516548,1.092087,0.796791,03:01
3,0.412751,1.126544,0.794118,02:58


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.436070,1.138099,0.795455,02:56
1,0.467533,1.092582,0.795455,03:01
2,0.375185,1.064078,0.807487,03:01
3,0.348429,1.041360,0.810160,03:13


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.339878,1.053229,0.804813,03:06
1,0.396267,1.010980,0.816845,03:13
2,0.323856,1.040165,0.823529,03:15
3,0.275835,1.027098,0.822192,03:08


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.275299,1.057188,0.812834,03:15
1,0.314572,1.118975,0.802139,03:12
2,0.273271,1.043661,0.819519,03:14
3,0.231971,1.014725,0.823529,02:58


In [41]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.241679,1.083959,0.827540,03:16
1,0.258431,1.136909,0.816845,03:14
2,0.229004,0.998223,0.832888,03:16
3,0.183843,1.014030,0.836898,03:06


In [42]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.195242,1.071312,0.830214,03:18
1,0.234679,1.136552,0.831551,03:19
2,0.214331,1.089261,0.831551,03:07
3,0.177349,1.076499,0.830214,03:13


In [43]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.198318,1.053751,0.828877,03:02
1,0.220889,1.103334,0.826203,03:12
2,0.192770,1.103213,0.835562,03:11
3,0.183565,1.125409,0.836898,02:59


In [44]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.188805,1.062185,0.834225,03:19
1,0.188646,1.106584,0.838235,03:12


In [45]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.189847,1.095021,0.840909,03:07
1,0.169196,1.072066,0.842246,03:09


In [46]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.175907,1.107557,0.835562,03:14


In [47]:
# save the best model

learn.save_encoder('pubmed_cadec_askpatient_mixed_fold6')

# Part three: Predict on the test dataset

In [48]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [49]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[2, 5, 8, 218, 8, 8, 8, 8, 8, 8, 8, 3, 146, 27, 31, 34, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 735, 41, 41, 41, 88, 49, 495, 49, 49, 100, 100, 100, 100, 100, 51, 53, 53, 53, 53, 53, 597, 56, 58, 58, 58, 286, 64, 67, 71, 71, 76, 76, 864, 182, 79, 318, 318, 318, 614, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 79, 478, 79, 108, 108, 108, 37, 495, 80, 80, 80, 80, 495, 81, 82, 83, 89, 218, 91, 91, 91, 91, 91, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 96, 96, 96, 100, 545, 100, 100, 100, 100, 286, 106, 108, 108, 108, 108, 864, 462, 111, 111, 111, 112, 113, 114, 116, 117, 117, 572, 243, 717, 122, 24, 125, 455, 125, 125, 125, 125, 125, 125, 125, 375, 127, 131, 135, 89, 63, 269, 151, 151, 427, 157, 157, 163, 165, 171, 172, 172, 172, 172, 172, 173, 71, 179, 182, 182, 182, 183, 183, 183, 183, 184, 184, 187, 187, 218, 189, 190, 190, 191, 197, 197, 202, 205, 210, 210, 211, 873, 214, 214, 214, 896, 896, 896, 896, 896, 896, 721, 214, 214

In [50]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

659
0.7609699769053118
